In [ ]:
import matplotlib.pyplot as plt
import os
import re
import string
import tensorflow as tf
import pandas as pd

from utils import predicted_test_data_to_result_csv
from keras import layers
from keras import losses

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
# print(os.environ['LD_LIBRARY_PATH'])
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
train_path = "data/base/goodreads_train.csv"
test_path = "data/base/goodreads_test.csv"

epochs = 65
model_nb = 2

max_features = 10000
sequence_length = 100

embedding_dim = 50
learning_rate = 0.0001
batch_size = 10000
dropout_rate = 0.0

In [30]:
df_train = pd.read_csv(train_path, sep=",")

KeyboardInterrupt: 

In [ ]:
index = df_train[(df_train['rating'] == 0)].index
df_train.drop(index, inplace=True)
df_train.reset_index(inplace=True, drop=True)

In [ ]:
target = df_train.pop('rating')
target = target - 1

In [ ]:
features = df_train["review_text"]

In [ ]:
all_data = tf.data.Dataset.from_tensor_slices((features, target)).shuffle(10, reshuffle_each_iteration=False)

In [ ]:
def is_test(x, y):
    return x % 4 == 0

def is_train(x, y):
    return not is_test(x, y)

recover = lambda x,y: y

raw_validation_dataset = all_data.enumerate() \
                    .filter(is_test) \
                    .map(recover)

raw_train_dataset = all_data.enumerate() \
                    .filter(is_train) \
                    .map(recover)

In [ ]:
raw_train_dataset = raw_train_dataset.batch(batch_size=batch_size)
raw_validation_dataset = raw_validation_dataset.batch(batch_size=batch_size)

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_spoilers = tf.strings.regex_replace(lowercase, '\*\* spoiler alert \*\*', ' ')
    return tf.strings.regex_replace(stripped_spoilers,
                                    '[%s]' % re.escape(string.punctuation),
                                    '')

In [ ]:
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [ ]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_dataset.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

validation_text = raw_validation_dataset.map(lambda x, y: x)
vectorize_layer.adapt(validation_text)

In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [ ]:
train_ds = raw_train_dataset.map(vectorize_text)
validation_ds = raw_validation_dataset.map(vectorize_text)

In [ ]:
model = tf.keras.Sequential()

model.add(layers.Embedding(max_features + 1, embedding_dim, input_length=sequence_length))

model.add(layers.Conv1D(64, 3, activation = 'relu', padding = 'same'))
model.add(layers.Conv1D(64, 3, activation = 'relu', padding = 'same'))
model.add(layers.MaxPooling1D(3)) # 2 => 3
# model.add(layers.Dropout(dropout_rate))

model.add(layers.Conv1D(128, 5, activation = 'relu', padding = 'same'))
model.add(layers.Conv1D(128, 5, activation = 'relu', padding = 'same'))
model.add(layers.MaxPooling1D(3))
# model.add(layers.Dropout(dropout_rate))

model.add(layers.Conv1D(256, 5, activation = 'relu', padding = 'same'))
model.add(layers.Conv1D(256, 5, activation = 'relu', padding = 'same'))
model.add(layers.MaxPooling1D(3))
# model.add(layers.Dropout(dropout_rate))

model.add(layers.Conv1D(512, 5, activation = 'relu', padding = 'same'))
model.add(layers.Conv1D(512, 5, activation = 'relu', padding = 'same'))
model.add(layers.MaxPooling1D(3))
# model.add(layers.Dropout(dropout_rate))

model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(32, activation = 'relu'))
# model.add(layers.Dropout(dropout_rate))
model.add(layers.Dense(5, activation = 'softmax'))

model.summary()

In [ ]:
model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
exp_name = f'conv_net_model_{model_nb}_lr_{learning_rate}_bs_{batch_size}_emb_{embedding_dim}_dr_{dropout_rate}'

In [ ]:
history = model.fit(
    train_ds,
    callbacks=[tf.keras.callbacks.TensorBoard("logs/" + exp_name)],
    validation_data=validation_ds,
    epochs=epochs)

In [ ]:
export_model = tf.keras.Sequential([
    vectorize_layer,
    model
])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy']
)

In [ ]:
df_test = pd.read_csv(test_path, sep=",")

df_test_modified = df_test.drop(columns=[
    'user_id',
    'book_id',
    'review_id',
    'date_added',
    'date_updated',
    'read_at',
    'started_at',
    'n_votes',
    'n_comments'
], inplace=False)


In [ ]:
# test_data_numpy = df_test.to_numpy()

predicted_test_data = export_model.predict(df_test_modified)

In [ ]:
df_test.head()

In [ ]:
predicted_test_data_to_result_csv(df_test, predicted_test_data, exp_name)

export_model.save("saved_model/embedding_model_1")